In [143]:
import pandas as pd
import numpy as np
import urllib2
import json

ImportError: No module named pyspark

In [2]:
od_matrix = pd.read_csv('od_matrix_05_11_2017.csv')

In [3]:
od_matrix.columns

Index([u'route', u'tripNum', u'shapeId', u'shapeSequence', u'shapeLat',
       u'shapeLon', u'distanceTraveledShape', u'busCode', u'gpsPointId',
       u'gpsLat', u'gpsLon', u'distanceToShapePoint', u'timestamp',
       u'stopPointId', u'problem', u'birthdate', u'cardTimestamp', u'lineName',
       u'gender', u'date', u'id', u'o_route', u'o_bus_code', u'o_date',
       u'o_tripNum', u'o_timestamp', u'o_shape_id', u'o_shape_seq',
       u'o_shape_lat', u'o_shape_lon', u'o_stop_id', u'o_boarding_id',
       u'cardNum', u'next_o_route', u'next_o_bus_code', u'next_o_date',
       u'next_o_tripNum', u'next_o_timestamp', u'next_o_shape_id',
       u'next_o_shape_seq', u'next_o_shape_lat', u'next_o_shape_lon',
       u'next_o_stop_id', u'next_o_boarding_id', u'o_unixtimestamp',
       u'next_o_unixtimestamp', u'leg_duration', u'dist', u'rn'],
      dtype='object')

In [4]:
od_matrix.head()

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,next_o_shape_seq,next_o_shape_lat,next_o_shape_lon,next_o_stop_id,next_o_boarding_id,o_unixtimestamp,next_o_unixtimestamp,leg_duration,dist,rn
0,374,2,1895,5950316,-25.428322,-49.266739,9309.107,CC171,NaN,-25.428408,...,4870329,-25.433636,-49.272494,26373,1262720385064,37193,63950,445.950000,0.826454,1
1,50,1,1719,5305699,-25.450914,-49.226300,14546.742,DR103,NaN,-25.450981,...,5304613,-25.450921,-49.226295,32410,377957122168,35939,66247,505.133333,0.000920,1
2,967,13,2286,6087787,-25.426956,-49.279220,1195.869,MC861,NaN,-25.426836,...,6086302,-25.426275,-49.277075,33734,1571958030378,61513,46571,-1.000000,0.228346,1
3,535,3,1979,6393158,-25.552426,-49.254353,0.000,EA600,NaN,-25.552370,...,6403979,-25.550383,-49.261133,31258,1382979469425,37652,56596,315.733333,0.717050,1
4,463,8,1915,6348024,-25.431822,-49.267690,274.616,DC087,NaN,NaN,...,5386363,-25.430770,-49.276242,29420,609885356066,55623,37323,-1.000000,0.866776,1


In [5]:
od_matrix['o_datetime'] = pd.to_datetime(od_matrix['date'] + ' ' + od_matrix['o_timestamp'])

In [6]:
od_matrix['o_base_timestamp'] = (od_matrix['o_datetime'] - pd.Timedelta('2 min')).dt.strftime('%H:%M:%S')

In [7]:
od_matrix[['o_datetime','o_base_timestamp']].head()

,o_datetime,o_base_timestamp
0,2017-05-10 07:19:53,07:17:53
1,2017-05-10 06:58:59,06:56:59
2,2017-05-10 14:05:13,14:03:13
3,2017-05-10 07:27:32,07:25:32
4,2017-05-10 12:27:03,12:25:03


In [8]:
simple_od_matrix = od_matrix[['date','route','o_stop_id','o_timestamp','o_shape_lat','o_shape_lon','stopPointId','timestamp','shapeLat','shapeLon']]

In [9]:
simple_od_matrix.head()

,date,route,o_stop_id,o_timestamp,o_shape_lat,o_shape_lon,stopPointId,timestamp,shapeLat,shapeLon
0,2017-05-10,374,31957,07:19:53,-25.413083,-49.229020,26362,07:41:47,-25.428322,-49.266739
1,2017-05-10,50,31740,06:58:59,-25.475134,-49.299069,31769,07:32:13,-25.450914,-49.226300
2,2017-05-10,967,26345,14:05:13,-25.429278,-49.271228,26722,14:07:44,-25.426956,-49.279220
3,2017-05-10,535,36880,07:27:32,-25.554885,-49.255697,4664,08:06:00,-25.552426,-49.254353
4,2017-05-10,463,26340,12:27:03,-25.432775,-49.270209,29922,12:34:16,-25.431822,-49.267690


In [10]:
otp_server_url = 'http://localhost:5601/otp/'
otp_http_request = 'routers/ctba/plan?fromPlace={},{}&toPlace={},{}&mode=TRANSIT,WALK&date={}&time={}'
def get_otp_itineraries(o_lat,o_lon,d_lat,d_lon,date,time):
    otp_request_url = otp_server_url + otp_http_request.format(o_lat,o_lon,d_lat,d_lon,date,time)
    return json.loads(urllib2.urlopen(otp_request_url).read())

In [12]:
test_itinerary = get_otp_itineraries(-25.413083,-49.229020,-25.428322,-49.266739,'05-10-2017','7:19:00')

print test_itinerary

{u'elevationMetadata': {u'geoidElevation': False, u'ellipsoidToGeoidDifference': 3.686111287840708}, u'plan': {u'date': 1494411540000, u'to': {u'lat': -25.428322, u'vertexType': u'NORMAL', u'lon': -49.266739, u'name': u'Destination', u'orig': u''}, u'itineraries': [{u'walkTime': 2, u'legs': [{u'distance': 0.151, u'from': {u'vertexType': u'NORMAL', u'name': u'Origin', u'lon': -49.22902, u'departure': 1494411566000, u'lat': -25.413083, u'orig': u''}, u'interlineWithPreviousLeg': False, u'transitLeg': False, u'realTime': False, u'route': u'', u'departureDelay': 0, u'agencyTimeZoneOffset': -10800000, u'to': {u'arrival': 1494411567000, u'vertexType': u'TRANSIT', u'name': u'Rua Fagundes Varela, 1295 - Jardim Social', u'stopSequence': 12, u'lon': -49.229029895014, u'departure': 1494411568000, u'stopId': u'1:31957', u'stopCode': u'130279', u'lat': -25.413059468751, u'stopIndex': 11}, u'rentedBike': False, u'arrivalDelay': 0, u'mode': u'WALK', u'startTime': 1494411566000, u'duration': 1.0, u'st

In [14]:
#simple_od_matrix.loc[:,'itineraries'] = simple_od_matrix.apply(
#    lambda row: get_otp_itineraries(row['o_shape_lat'], row['o_shape_lon'], row['shapeLat'], row['shapeLon'],row['date'],row['o_timestamp']), axis=1)

In [15]:
#simple_od_matrix.head()

### New Approach

In [16]:
trips_otp_response = {}
counter = 0
for index, row in od_matrix[:10].iterrows():
        id=long(row['o_boarding_id'])
        trip_plan = get_otp_itineraries(row['o_shape_lat'], row['o_shape_lon'], row['shapeLat'], row['shapeLon'],row['date'],row['o_base_timestamp'])
        trips_otp_response[id] = trip_plan
        counter+=1

In [137]:
trips_plans = []

for trip in trips_otp_response.keys():
    itinerary_id = 1
    for itinerary in trips_otp_response[trip]['plan']['itineraries']:
        leg_id = 1
        for leg in itinerary['legs']:
            route = leg['route'] if leg['route'] != '' else None
            fromStopId = leg['from']['stopId'].split(':')[1] if leg['mode'] == 'BUS' else None
            toStopId = leg['to']['stopId'].split(':')[1] if leg['mode'] == 'BUS' else None
            start_time = pd.to_datetime(long(leg['startTime']),unit='ms').tz_localize('UTC').tz_convert('America/Sao_Paulo')
            end_time = pd.to_datetime(long(leg['endTime']),unit='ms').tz_localize('UTC').tz_convert('America/Sao_Paulo')
            duration = end_time - start_time
            date_str = start_time.strftime('%Y-%m-%d')
            start_time_str = start_time.strftime('%H:%M:%S')
            end_time_str = end_time.strftime('%H:%M:%S')
            trips_plans.append((date_str,trip,itinerary_id,leg_id,start_time_str,end_time_str,leg['mode'],route,fromStopId,toStopId, duration))
            leg_id += 1
        itinerary_id += 1

labels=['date','trip_id','itinerary_id','leg_id','start_time','end_time','mode','route','from_stop_id','to_stop_id','duration']
trips_plans_df = pd.DataFrame.from_records(trips_plans,columns=labels)
trips_plans_df['route'] = trips_plans_df['route'].astype(float)
trips_plans_df['from_stop_id'] = trips_plans_df['from_stop_id'].astype(float)
trips_plans_df['to_stop_id'] = trips_plans_df['to_stop_id'].astype(float)

In [138]:
trips_plans_df

,date,trip_id,itinerary_id,leg_id,start_time,end_time,mode,route,from_stop_id,to_stop_id,duration
0,2017-05-10,1434519077089,1,1,07:19:26,07:19:27,WALK,NaN,NaN,NaN,00:00:01
1,2017-05-10,1434519077089,1,2,07:19:28,07:42:00,BUS,374.0,31957.0,26362.0,00:22:32
2,2017-05-10,1434519077089,1,3,07:42:01,07:42:02,WALK,NaN,NaN,NaN,00:00:01
3,2017-05-10,1434519077089,2,1,07:27:14,07:27:15,WALK,NaN,NaN,NaN,00:00:01
4,2017-05-10,1434519077089,2,2,07:27:16,07:48:00,BUS,361.0,31957.0,26361.0,00:20:44
5,2017-05-10,1434519077089,2,3,07:48:01,07:48:17,WALK,NaN,NaN,NaN,00:00:16
6,2017-05-10,1434519077089,3,1,07:35:11,07:35:12,WALK,NaN,NaN,NaN,00:00:01
7,2017-05-10,1434519077089,3,2,07:35:13,07:59:00,BUS,374.0,31957.0,26362.0,00:23:47
8,2017-05-10,1434519077089,3,3,07:59:01,07:59:02,WALK,NaN,NaN,NaN,00:00:01
9,2017-05-10,249108103395,1,1,16:36:21,16:36:32,WALK,NaN,NaN,NaN,00:00:11


In [127]:
trips_plans_df.describe(include='all')

,date,trip_id,itinerary_id,leg_id,start_time,end_time,mode,route,from_stop_id,to_stop_id,duration
count,84,8.400000e+01,84.000000,84.000000,84,84,84,27.000000,27.000000,27.000000,84
unique,1,NaN,NaN,NaN,84,84,2,NaN,NaN,NaN,NaN
top,05-10-2017,NaN,NaN,NaN,07:19:28,07:26:54,WALK,NaN,NaN,NaN,NaN
freq,84,NaN,NaN,NaN,1,1,57,NaN,NaN,NaN,NaN
mean,NaN,1.085809e+12,2.047619,1.964286,NaN,NaN,NaN,556.592593,29924.037037,28627.814815,0 days 00:06:51.797619
std,NaN,5.099357e+11,0.820002,0.827923,NaN,NaN,NaN,323.680191,6292.226287,7718.793932,0 days 00:12:16.581975
min,NaN,2.491081e+11,1.000000,1.000000,NaN,NaN,NaN,40.000000,14298.000000,4664.000000,0 days 00:00:01
25%,NaN,5.841156e+11,1.000000,1.000000,NaN,NaN,NaN,363.000000,28520.000000,26362.000000,0 days 00:00:07
50%,NaN,9.534827e+11,2.000000,2.000000,NaN,NaN,NaN,535.000000,31957.000000,30052.000000,0 days 00:00:48
75%,NaN,1.477469e+12,3.000000,3.000000,NaN,NaN,NaN,860.000000,33681.000000,33375.000000,0 days 00:05:02.750000


### Read Bus Data

In [128]:
bus_trips_data = pd.read_csv('buste_crowd.csv')

In [129]:
bus_trips_data.columns

Index([u'route', u'tripNum', u'shapeId', u'shapeSequence', u'shapeLat',
       u'shapeLon', u'distanceTraveledShape', u'busCode', u'gpsPointId',
       u'gpsLat', u'gpsLon', u'distanceToShapePoint', u'timestamp',
       u'stopPointId', u'problem', u'birthdate', u'cardTimestamp', u'lineName',
       u'gender', u'date', u'id', u'boarding_cnt', u'alighting_cnt',
       u'crowd_bal', u'num_pass', u'extrap_factor', u'ext_num_pass'],
      dtype='object')

In [130]:
bus_trips_data['stopPointId'] = bus_trips_data['stopPointId'].astype(float)

In [131]:
bus_trips_data.head()

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,lineName,gender,date,id,boarding_cnt,alighting_cnt,crowd_bal,num_pass,extrap_factor,ext_num_pass
0,463.0,1,2846,6348468,-25.478025,-49.203033,1128.772,DC087,NaN,-25.478023,...,NaN,NaN,2017-05-10,1279900255353,0,0,0,0,39.22807,0.0
1,463.0,1,2846,6348493,-25.474027,-49.209193,1899.853,DC087,NaN,NaN,...,NaN,NaN,2017-05-10,549755815049,0,0,0,0,39.22807,0.0
2,463.0,1,2846,6348516,-25.468650,-49.213528,2638.152,DC087,NaN,NaN,...,NaN,NaN,2017-05-10,523986011178,0,0,0,0,39.22807,0.0
3,463.0,1,2846,6348531,-25.464090,-49.217170,3262.176,DC087,NaN,-25.463931,...,NaN,NaN,2017-05-10,180388627571,0,0,0,0,39.22807,0.0
4,463.0,1,2846,6348545,-25.459453,-49.220872,3896.639,DC087,NaN,-25.459610,...,NaN,NaN,2017-05-10,1142461301906,0,0,0,0,39.22807,0.0


In [140]:
real_trip_beginning = pd.merge(trips_plans_df,bus_trips_data, left_on=['date','route','from_stop_id'],right_on=['date','route','stopPointId'],how='inner')
real_trip_beginning = real_trip_beginning.groupby(['date','route','trip_id','itinerary_id','leg_id','from_stop_id'])

In [142]:
real_trip_beginning[['date','route','from_stop_id','tripNum','timestamp','start_time']]

,date,route,from_stop_id,tripNum,timestamp,start_time
0,2017-05-10,374.0,31957.0,1,06:24:44,07:19:28
1,2017-05-10,374.0,31957.0,2,07:19:18,07:19:28
2,2017-05-10,374.0,31957.0,3,08:22:47,07:19:28
3,2017-05-10,374.0,31957.0,4,09:39:23,07:19:28
4,2017-05-10,374.0,31957.0,5,10:36:29,07:19:28
5,2017-05-10,374.0,31957.0,6,11:37:44,07:19:28
6,2017-05-10,374.0,31957.0,7,12:45:21,07:19:28
7,2017-05-10,374.0,31957.0,8,13:52:44,07:19:28
8,2017-05-10,374.0,31957.0,9,15:03:10,07:19:28
9,2017-05-10,374.0,31957.0,10,16:05:30,07:19:28


In [168]:
trips_plans_df.head()

,trip_id,itinerary_id,leg_id,start_time,end_time,mode,route,from_stop_id,to_stop_id,duration
0,1434519077089,1,1,1494411566000,1494411567000,WALK,,None,None,00:00:01
1,1434519077089,1,2,1494411568000,1494412920000,BUS,374,31957,26362,00:22:32
2,1434519077089,1,3,1494412921000,1494412922000,WALK,,None,None,00:00:01
3,1434519077089,2,1,1494412034000,1494412035000,WALK,,None,None,00:00:01
4,1434519077089,2,2,1494412036000,1494413280000,BUS,361,31957,26361,00:20:44


In [173]:
for index, row in trips_plans_df.iterrows():
    if row['mode'] == 'BUS':
        bus_trips_data.filter((bus_trips_data.route == row['route']) & 
                              (int(bus_trips_data.stopPointId) == int(row['from_stop_id']))
    

SyntaxError: invalid syntax (<ipython-input-173-9376cbc206d6>, line 5)